In [12]:
import os
import json
import geojson
import pandas as pd
import pymongo
from pprint import pprint
import numpy as np
from flask import Flask, render_template, jsonify

In [2]:
def path_constructor(parent, child):
    return os.path.join(parent, child)

In [7]:
def pathList(fileList,root):
    path=[]
    for f in fileList:
        t=path_constructor(root, f)
        path.append(t)
    return path

In [5]:
# data folder
dataFolder = path_constructor("static", "data")
jsonFolder=path_constructor(dataFolder,"json")
csvFolder=path_constructor(dataFolder,"csv")
jsonFiles = os.listdir(jsonFolder)

In [8]:
# json files path
jsonFilespath = pathList(jsonFiles, jsonFolder)

In [18]:
jsonFilespath

['static\\data\\json\\American_Goldfinch.json',
 'static\\data\\json\\American_Robin.json',
 'static\\data\\json\\Barn_Swallow.json',
 'static\\data\\json\\Blue-grey_Gnatcatcher.json',
 'static\\data\\json\\Blue_Jay.json',
 'static\\data\\json\\Broad-winged_Hawk.json',
 'static\\data\\json\\Carolina_Chickadee.json',
 'static\\data\\json\\Carolina_Wren.json',
 'static\\data\\json\\Cedar_Waxwing.json',
 'static\\data\\json\\Eastern_Bluebird.json',
 'static\\data\\json\\Northern_Cardinal.json',
 'static\\data\\json\\Rose-breasted_Grosbeak.json',
 'static\\data\\json\\Ruby-crowned_Kinglet.json',
 'static\\data\\json\\Tufted_Titmouse.json']

In [10]:
# Audio files used in the ML project
bird_df = pd.read_csv(path_constructor(csvFolder, "bird_calls.csv"))
bird_df.head()

,fname,label
0,American_Goldfinch_XC114342.wav,American_Goldfinch
1,American_Goldfinch_XC124312.wav,American_Goldfinch
2,American_Goldfinch_XC133564.wav,American_Goldfinch
3,American_Goldfinch_XC133565.wav,American_Goldfinch
4,American_Goldfinch_XC141469.wav,American_Goldfinch


In [11]:
# file IDs
birdID = bird_df.fname.str[-10:-4]
birdList=list(np.unique(bird_df.label))

In [13]:
# Set up mongo connection
client = pymongo.MongoClient("mongodb://localhost:27017")

In [15]:
db=client["Bird_db"]

In [16]:
collection=db["BirdCalls"]

In [30]:
# Function to insert the json data in a collection
def insertData(pathList, collection):    
    documents = []
    for p in pathList:
        with open(p, 'r', encoding='utf-8') as f:
            contents = json.load(f)
            documents.append(contents)
    collection.insert_many(documents)

In [31]:
insertData(jsonFilespath, collection)

In [32]:
client.list_database_names()

['Bird_db',
 'Migratory_Birds',
 'admin',
 'classDB',
 'config',
 'craigslist_app',
 'local',
 'mars_mission_db',
 'travel_db']

In [33]:
db.list_collection_names()

['BirdCalls']

In [39]:
def queryBirds(birdName, birdCollection):
    birdData = [b for b in birdCollection.find({"recordings":[{"en": birdName}]})]          
    return birdData

In [50]:
pprint(collection.find_one())

{'_id': ObjectId('603d6a694525d21f984d6956'),
 'numPages': 1,
 'numRecordings': '130',
 'numSpecies': '1',
 'page': 1,
 'recordings': [{'also': ['Poecile atricapillus'],
                 'alt': '0',
                 'bird-seen': 'yes',
                 'cnt': 'United States',
                 'date': '2019-08-31',
                 'en': 'American Goldfinch',
                 'file': '//www.xeno-canto.org/543352/download',
                 'file-name': 'XC543352-AMGO_2019-08-31_Smith_Bybee_Wetlands_North_Portland_OR_1531.mp3',
                 'gen': 'Spinus',
                 'id': '543352',
                 'lat': '45.6123',
                 'length': '1:02',
                 'lic': '//creativecommons.org/licenses/by-nc-sa/4.0/',
                 'lng': '-122.7352',
                 'loc': 'Smith Bybee Wetlands, North Portland, Multnomah '
                        'County, Oregon',
                 'playback-used': 'no',
                 'q': 'no score',
                 'rec': 'Thomas

                 'sono': {'full': '//www.xeno-canto.org/sounds/uploaded/RFTXRYBVBX/ffts/XC486563-full.png',
                          'large': '//www.xeno-canto.org/sounds/uploaded/RFTXRYBVBX/ffts/XC486563-large.png',
                          'med': '//www.xeno-canto.org/sounds/uploaded/RFTXRYBVBX/ffts/XC486563-med.png',
                          'small': '//www.xeno-canto.org/sounds/uploaded/RFTXRYBVBX/ffts/XC486563-small.png'},
                 'sp': 'tristis',
                 'ssp': '',
                 'time': '06:37',
                 'type': 'flight call',
                 'uploaded': '2019-07-11',
                 'url': '//www.xeno-canto.org/486563'},
                {'also': ['Agelaius phoeniceus'],
                 'alt': '180',
                 'bird-seen': 'yes',
                 'cnt': 'Canada',
                 'date': '2019-05-17',
                 'en': 'American Goldfinch',
                 'file': '//www.xeno-canto.org/476269/download',
                 'file-name':

                 'lat': '41.3977',
                 'length': '1:43',
                 'lic': '//creativecommons.org/licenses/by-nc-sa/4.0/',
                 'lng': '-91.4',
                 'loc': 'Cone Marsh State WMA, Louisa County, Iowa',
                 'playback-used': 'no',
                 'q': 'B',
                 'rec': 'Paul Marvin',
                 'rmk': 'pair, one was folling another and begging,  it called '
                        'both from tree branches and in flight\r\n'
                        '\r\n'
                        'I think (but not sure) the begging bird was calling',
                 'sono': {'full': '//www.xeno-canto.org/sounds/uploaded/RFTXRYBVBX/ffts/XC210746-full.png',
                          'large': '//www.xeno-canto.org/sounds/uploaded/RFTXRYBVBX/ffts/XC210746-large.png',
                          'med': '//www.xeno-canto.org/sounds/uploaded/RFTXRYBVBX/ffts/XC210746-med.png',
                          'small': '//www.xeno-canto.org/sounds/upl

                 'alt': '',
                 'bird-seen': 'unknown',
                 'cnt': 'Canada',
                 'date': '2012-07-07',
                 'en': 'American Goldfinch',
                 'file': '//www.xeno-canto.org/158924/download',
                 'file-name': 'XC158924-IAC 2012-07-07 T06-44-32.mp3',
                 'gen': 'Spinus',
                 'id': '158924',
                 'lat': '48.576806',
                 'length': '0:56',
                 'lic': '//creativecommons.org/licenses/by-nc-nd/3.0/',
                 'lng': '-123.37179',
                 'loc': 'Island View Beach Regional Park, Central Saanich, '
                        'Vancouver Island, BC',
                 'playback-used': 'no',
                 'q': 'B',
                 'rec': 'Ian Cruickshank',
                 'rmk': 'adult\n'
                        '\n'
                        'Recorded on a Marantz PMD661 recorder using a '
                        'Sennheiser ME66 microphone.',
  

                 'lat': None,
                 'length': '0:30',
                 'lic': '//creativecommons.org/licenses/by-nc-sa/4.0/',
                 'lng': None,
                 'loc': 'Croton Point Park, Croton, New York',
                 'playback-used': 'no',
                 'q': 'C',
                 'rec': 'Daniel Parker',
                 'rmk': '',
                 'sono': {'full': '//www.xeno-canto.org/sounds/uploaded/AMMGWAIZWI/ffts/XC268738-full.png',
                          'large': '//www.xeno-canto.org/sounds/uploaded/AMMGWAIZWI/ffts/XC268738-large.png',
                          'med': '//www.xeno-canto.org/sounds/uploaded/AMMGWAIZWI/ffts/XC268738-med.png',
                          'small': '//www.xeno-canto.org/sounds/uploaded/AMMGWAIZWI/ffts/XC268738-small.png'},
                 'sp': 'tristis',
                 'ssp': '',
                 'time': '19:30',
                 'type': 'begging call, juvenile',
                 'uploaded': '2015-08-17',
         

In [52]:
pprint(collection.find_one({"gen":"Spinus"}))

None


In [ ]:
for c in collection.find_one("")